In [53]:
import tensorflow as tf
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import losses, optimizers
from tensorflow.keras import backend as K

img_rows, img_cols = 28, 28
# 通过Keras封装好的API加载MNIST数据。其中trainX就是一个60000 * 28 * 28的数组，
# trainY是每一张图片对应的数字。
def load_data(path='/Users/zhangjrjlu/datasets/mnist.npz'):       #####该部分是对于.npz(numpy格式)文件的读取方法
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)
 
# the data, split between train and test sets
(trainX, trainY), (testX, testY) = load_data()

# 根据对图像编码的格式要求来设置输入层的格式。需要根据对于图像格式的要求来判断
if K.image_data_format() == 'channels_first':     
    trainX = trainX.reshape(trainX.shape[0], 1, img_rows, img_cols)   #四维数组，第一项是图片的数量，表示第几张图片
    testX = testX.reshape(testX.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    trainX = trainX.reshape(trainX.shape[0], img_rows, img_cols, 1)
    testX = testX.reshape(testX.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0
trainY = keras.utils.to_categorical(trainY, 10)
testY = keras.utils.to_categorical(testY, 10)
x = testX
y = testY

In [42]:
def convert(trainX, trainY, testX, testY):
    #把np格式转换为张量格式
    trainX = tf.convert_to_tensor(trainX, dtype=tf.float32)
    trainY = tf.convert_to_tensor(trainY, dtype=tf.int32)
    testX = tf.convert_to_tensor(testX, dtype=tf.float32)
    testY = tf.convert_to_tensor(testY, dtype=tf.int32)
    
#     trainY = tf.cast(trainY, dtype=tf.int32)
#     testY = tf.cast(testY, dtype=tf.int32)
#     trainY = tf.one_hot(trainY, depth=10)
#     testY = tf.one_hot(testY, depth=10)
    
    return trainX, trainY, testX, testY

trainX, trainY, testX, testY = convert(trainX, trainY, testX, testY)


In [43]:
model = Sequential()

model.add(Conv2D(6, kernel_size=3, strides=1, activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(16, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10))
"""
如果想要在这里使用激活函数可以使用model.add(Dense(10, activation='softmax'))
因为当前程序为了使用tensorflow的方法实现，因此不再这里进行激活
"""
model.build(input_shape=(60000, 28, 28, 1))
"""
使用Sequential()中的build函数进行初始化必须是四维，第一维度是输入的数量
在model.layers中初始化，只能初始化图片的维度三维，可以是channel first，也可以是channel last
"""
#model.build(input_shape=input_shape)
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 16)        880       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 120)               48120     
_________________________________________________________________
dense_28 (Dense)             (None, 84)               

In [58]:
cross_entropy = losses.CategoricalCrossentropy(from_logits=True)
"""
losses.CategoricalCrossentropy是交叉熵损失函数类，当参数from_logits为True的时候，会把softmax激活函数实现在损失函数中
也可以直接使用
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.SGD(),metrics=['accuracy'])
直接把把损失函数，优化函数以及评测方法写入
"""
def train(cross_entropy, X, Y, model):
    with tf.GradientTape() as tape:
        y_ = model(X)
        loss = cross_entropy(Y, y_)
    grad = tape.gradient(loss, model.trainable_variables)
    optimizers.Adam().apply_gradients(zip(grad, model.trainable_variables))

for i in range(40):
    train(cross_entropy, trainX, trainY, model)



In [59]:
c, t = 0, 0
out = model(testX)
loss = cross_entropy(testY, out)
print(loss)
correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(testY, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy)


tf.Tensor(0.54380435, shape=(), dtype=float32)
tf.Tensor(0.8396, shape=(), dtype=float32)


In [23]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])
model.fit(trainX, trainY,
          batch_size=128,
          epochs=10,
          validation_data=(testX, testY))  
score = model.evaluate(testX, testY)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 114us/sample - loss: 1.6203 - accuracy: 0.5160 - val_loss: 0.5194 - val_accuracy: 0.8513
Epoch 2/10
60000/60000 [==============================] - 6s 107us/sample - loss: 0.4034 - accuracy: 0.8780 - val_loss: 0.3155 - val_accuracy: 0.9037
Epoch 3/10
60000/60000 [==============================] - 7s 111us/sample - loss: 0.2776 - accuracy: 0.9140 - val_loss: 0.2476 - val_accuracy: 0.9208
Epoch 4/10
60000/60000 [==============================] - 7s 111us/sample - loss: 0.2174 - accuracy: 0.9334 - val_loss: 0.1804 - val_accuracy: 0.9407
Epoch 5/10
60000/60000 [==============================] - 6s 108us/sample - loss: 0.1799 - accuracy: 0.9456 - val_loss: 0.1539 - val_accuracy: 0.9540
Epoch 6/10
60000/60000 [==============================] - 7s 109us/sample - loss: 0.1533 - accuracy: 0.9534 - val_loss: 0.1406 - val_accuracy: 0.9554
Epoch 7/10
60000/60000 [==========================